# Storage penetration playbook

In [18]:
import psycopg2
import warehouse
from datetime import datetime, date
import os
import pwn

[*] Checking for new versions of pwntools
    To disable this functionality, set the contents of /home/yuri/.cache/.pwntools-cache-3.11/update to 'never' (old way).
    Or add the following lines to ~/.pwn.conf or ~/.config/pwn.conf (or /etc/pwn.conf system-wide):
        [update]
        interval=never
[*] You have the latest version of Pwntools (4.13.1)


 ### Put machines in the DB

In [2]:
try:
    warehouse.put_machine_with_id(1, "machine1", machine_line=1)
    warehouse.put_machine_with_id(2, "machine2", machine_line=0)
    warehouse.put_machine_with_id(3, "machine3")
    warehouse.put_machine_with_id(4, "machine4")
    warehouse.put_machine_with_id(5, "machine5")
except psycopg2.errors.UniqueViolation as e:
    print(e)

Executing query: INSERT INTO warehouse.machines (machineid,name,line) VALUES (1, 'machine1', 1) Returning machineid
duplicate key value violates unique constraint "machines_pkey"
DETAIL:  Key (machineid)=(1) already exists.



In [3]:
host = os.environ["POSTGRES_HOST"] if "POSTGRES_HOST" in os.environ else "localhost"

conn = psycopg2.connect(
    host=host,
    database="postgres",
    user="postgres",
    password="password",
    port="5432",
)

# Setting auto commit false
conn.autocommit = True
global cursor
# Creating a cursor object using the cursor() method
cursor = conn.cursor()

In [ ]:
from psycopg2.extensions import AsIs

machine_id = "20"
machine_name = "1); OR 1 = 1 -- "
# machine_type = "'); DROP TABLE warehouse.machines; --('"
columns = ["machineid", "name"]
values = [machine_id, machine_name]
query = "INSERT INTO warehouse.machines (%s) VALUES %s Returning machineid"
params = (AsIs(",".join(columns)), tuple(values))
print(cursor.mogrify(query, params).decode('utf-8'))

INSERT INTO warehouse.machines (machineid,name) VALUES ('20', '1); OR 1 = 1 -- ') Returning machineid


In [2]:
# put_machine_with_id(machine_id, machine_name, machine_type=None, machine_line=None, machine_factory=None)

try:
    warehouse.put_machine_with_id(machine_id="999", machine_name="1")
except Exception as e:
    print(e)

name 'warehouse' is not defined


### Querying for machine data

In [6]:
machine = warehouse.get_machine_data_by_name("machine1")[0]
print("Machine ID, Name, Line, Factory, Type")
print(machine)
machine_id = machine[0]

Machine ID, Name, Line, Factory, Type
(1, 'machine1', 1, 'Unknown', 'Unknown')


### Querying for measurments

In [14]:
measurements = warehouse.get_measurement_with_kpi(
    1,
    "cost",
    start_time=date(2020, 1, 1),
    end_time="2024-12-06 OR 1 = 1",
)

Executing query: SELECT * FROM warehouse.sensors_data WHERE machineid = 1 AND measurements->'cost' IS NOT NULL AND timestamp >= 2020-01-01 AND timestamp <= 2024-12-06 OR 1 = 1


InvalidDatetimeFormat: invalid input syntax for type timestamp: "2024-12-06 OR 1 = 1"
LINE 1: ... timestamp >= '2020-01-01'::date AND timestamp <= '2024-12-0...
                                                             ^


In [23]:
measurements = warehouse.get_measurement_with_kpi(
    "meow",
    "cost",
    start_time="2020-01-01",
)

Executing query: SELECT * FROM warehouse.sensors_data WHERE machineid = meow AND measurements->'cost' IS NOT NULL AND timestamp >= 2020-01-01


InvalidTextRepresentation: invalid input syntax for type integer: "meow"
LINE 1: ...T * FROM warehouse.sensors_data WHERE machineid = 'meow' AND...
                                                             ^


In [22]:
for mesurement in measurements:
    print(mesurement)

### Querying alerts

... by machine that caused the alert

In [19]:
machine_id = "A"*1000
alerts = warehouse.get_alert_by_machine_id(machine_id)
for alert in alerts:
    print(alert)

InvalidTextRepresentation: invalid input syntax for type integer: "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

... by production line

In [ ]:
# get alerts by line
alerts = warehouse.get_alerts_by_line(1)
for alert in alerts:
    print(alert)